In [ ]:
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git
%cd TransNetV2/inference

In [ ]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2

# Parse video info

In [3]:
videos_dir = './AIC_Video'
all_video_paths = dict()
for part in sorted(os.listdir(videos_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_video_paths[data_part] =  dict()

for data_part in sorted(all_video_paths.keys()):
    data_part_path = f'{videos_dir}/Videos_{data_part}/video'
    video_paths = sorted(os.listdir(data_part_path))
    video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
    for video_id, video_path in zip(video_ids, video_paths):
        video_path_full = f'{data_part_path}/{video_path}'
        all_video_paths[data_part][video_id] = video_path_full

# Inference

In [ ]:
model = TransNetV2()

save_dir = './SceneJSON'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for key, video_paths_dict in tqdm(all_video_paths.items()):
    video_ids = sorted(video_paths_dict.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_path = video_paths_dict[video_id]
        _, single_frame_predictions, _ = model.predict_video(video_path)
        
        # Generate list of scenes from predictions, returns tuples of (start frame, end frame)
        scenes = model.predictions_to_scenes(single_frame_predictions)
        
        with open(f"{save_dir}/{key}/{video_id}.json", 'w') as f:
            json.dump(scenes.tolist(), f)

In [8]:
!rm -r ./TransNetV2